* 라이브러리 설치

In [5]:
import pandas as pd
import numpy as np
import sklearn.impute
import sklearn.linear_model
import missingno as msno

* 데이터 불러오기

In [6]:
df_test = pd.read_csv('data/test.csv')
df_train = pd.read_csv('data/train.csv')

* 결측치 처리 함수 정의

In [7]:
def impute_missing(df) :
    df_imputed = df.copy()
    df_num = df.select_dtypes(include = 'number')
    df_cat = df.select_dtypes(exclude = 'number')
    
    df_imputed[df_num.columns] = sklearn.impute.SimpleImputer(strategy = 'mean').fit_transform(df_num)
    df_imputed[df_cat.columns] = sklearn.impute.SimpleImputer(strategy = 'most_frequent').fit_transform(df_cat)
    
    return df_imputed

* 자료 정리

In [8]:
set(df_train.columns) - set(df_test.columns)

{'Survived'}

In [9]:
df_train.select_dtypes(exclude = 'number').columns

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

In [10]:
X = pd.get_dummies(impute_missing(df_train).drop(['Survived', 'Name', 'Ticket', 'Cabin'], axis = 1))
y = df_train.Survived

In [11]:
X

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1.0,3.0,22.000000,1.0,0.0,7.2500,0,1,0,0,1
1,2.0,1.0,38.000000,1.0,0.0,71.2833,1,0,1,0,0
2,3.0,3.0,26.000000,0.0,0.0,7.9250,1,0,0,0,1
3,4.0,1.0,35.000000,1.0,0.0,53.1000,1,0,0,0,1
4,5.0,3.0,35.000000,0.0,0.0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,887.0,2.0,27.000000,0.0,0.0,13.0000,0,1,0,0,1
887,888.0,1.0,19.000000,0.0,0.0,30.0000,1,0,0,0,1
888,889.0,3.0,29.699118,1.0,2.0,23.4500,1,0,0,0,1
889,890.0,1.0,26.000000,0.0,0.0,30.0000,0,1,1,0,0


* predictor 생성 및 학습
  

In [12]:
predictr = sklearn.linear_model.LogisticRegression()

In [13]:
predictr.fit(X, y)

C:\Users\hollyriver\anaconda3\envs\ssk\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### **<span style='background-color: #fff700'>평가</span>**

In [14]:
predictr.score(X,y)

0.8002244668911336

> **여기 부분에서 점수가 강의노트에서보다 조금 더 낮게 나옵니다.**
>
> **게다가 해당 점수는 결측치를 대체하지 않았을 때의 점수와 동일합니다.**
>
> **그런데 해당 학습 모델을 적용시켜 캐글에 제출하면 정확도가 오히려 더 낮아집니다.**

* 제출

In [15]:
X_test = pd.get_dummies(impute_missing(df_test).drop(['Name', 'Ticket', 'Cabin'], axis = 1))
X_test

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892.0,3.0,34.50000,0.0,0.0,7.8292,0,1,0,1,0
1,893.0,3.0,47.00000,1.0,0.0,7.0000,1,0,0,0,1
2,894.0,2.0,62.00000,0.0,0.0,9.6875,0,1,0,1,0
3,895.0,3.0,27.00000,0.0,0.0,8.6625,0,1,0,0,1
4,896.0,3.0,22.00000,1.0,1.0,12.2875,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
413,1305.0,3.0,30.27259,0.0,0.0,8.0500,0,1,0,0,1
414,1306.0,1.0,39.00000,0.0,0.0,108.9000,1,0,1,0,0
415,1307.0,3.0,38.50000,0.0,0.0,7.2500,0,1,0,0,1
416,1308.0,3.0,30.27259,0.0,0.0,8.0500,0,1,0,0,1


In [73]:
df_test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [84]:
submit_csv = df_test[['PassengerId']].assign(Survived = predictr.predict(X_test))

In [87]:
submit_csv.to_csv('Logistic_impute.csv', index = False)

> Public Score : 0.75119 ...
